In [ ]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, data_handling, timeframes
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter



In [ ]:
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./half_month_collections.pkl")

In [ ]:
markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())

In [ ]:
def get_possible_dates(date : pd.Timestamp):
    """ Function to get the possible dates for placing a bid given the current date

    Args:
        date (pd.Timestamp): the current date

    Returns:
        (pd.date_range, str): the possible dates for placing a bid and for which market
    """
    if date.hour == 17: # FCR D-2
        return (pd.date_range(date + timedelta(days=1) + timedelta(hours=7), date + timedelta(days = 2) + timedelta(hours = 6), freq='H', tz = "Europe/Oslo"), "D_2")
    elif date.hour == 7: # aFRR
        return (pd.date_range(date + timedelta(hours = 17), date + timedelta(days = 1) + timedelta(hours = 16), freq='H', tz = "Europe/Oslo"), "aFRR")
    elif date.hour == 18: # FCR D-1
        return (pd.date_range(date + timedelta(hours=6), date + timedelta(days = 1) + timedelta(hours = 5), freq='H', tz = "Europe/Oslo"), "D_1")
    else:
        return ([], "No bids")

In [ ]:
def get_expected_prices_for_hours(possible_hours : pd.Timestamp, area : str, direction : str):
    """ Function to find the expected price for a given hour, area and direction. This function should be updated to be smarter in the future.

    Args:
        hour (pd.Timestamp): hour for which the expected price is calculated
        area (str): area for which the expected price is calculated
        direction (str): direction for which the expected price is calculated

    Returns:
        float: expected price for the given hour, area and direction
    """
    markets_to_check = [market for market in markets_name_dict.values() if market.area == area and market.direction == direction]
    return np.array([np.mean([market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in markets_to_check]) for hour in possible_hours])
    

In [ ]:
def place_bids(possible_hours : [pd.Timestamp], available_assets : dict, market : final_markets.ReserveMarket):
    """ Function to place bids for a given market and set of hours. The bids are placed for every hour in the set of hours. The bids are placed for the assets that are not already bid in to other markets in the given hours. The bids are placed for the assets that are compatible with the given market.

    Args:
        possible_hours (pd.Timestamp]): The time stamps for which the bids are placed
        available_assets (dict): Dictionary with the available assets for each hour
        market (final_markets.ReserveMarket): the market to be bid in to

    Returns:
        tuple (np.array, np.array): The possible revenue of the placed bids for each hour and the assets which are bid for each hour
    """
    expected_prices = get_expected_prices_for_hours(possible_hours, market.area, market.direction)
    hourly_assets = [available_assets[hour] for hour in possible_hours]
    #print(f"expected_pries: {expected_pries}")
    #print(f"hourly_assets: {hourly_assets}")
    if market.direction == "up":
        # check the compatibility for the assets
        possible_assets = [[asset for asset in hourly_assets[h] if asset.area == market.area and asset.direction != "down"] for h in range(len(possible_hours))]
        #print(f"possible_assets: {possible_assets}")
        #get the volumes in the correct direction
        possible_volumes = [sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        max_vol = np.array([market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for hour in possible_hours]) # set of volumes for markets
        possible_volumes = np.where(possible_volumes > [market.min_volume for _ in range(len(possible_volumes))], possible_volumes, 0)
        possible_volumes = np.where(possible_volumes <= max_vol, possible_volumes, 0)


        # get the prices for the given market within the given hours
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        # Calculate the possible revenues
        #print(f"possible_volumes: {possible_volumes}")
        #print(f"possible_prices: {possible_prices}")
        bids_to_be_made = np.where(possible_prices >= expected_prices, possible_prices, 0)
        possible_revenues = np.sum(possible_volumes * bids_to_be_made, axis = 1)
        # Place the bids in the dictionary
        # Must add a better way to store the used assets to the bid
        possible_assets = [assets if possible_revenues[index] > 0 else [] for index, assets in enumerate(possible_assets) ]     

        return (possible_revenues,possible_assets)
    elif market.direction == "down":
        possible_assets = [[asset for asset in hourly_assets[h] if asset.area == market.area and asset.direction != "up"] for h in range(len(possible_hours))]
        possible_volumes = [sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        # all possible volumes has to be higher than the minimum volume for the market
        possible_volumes = np.where(possible_volumes > [market.min_volume for _ in range(len(possible_volumes))], possible_volumes, 0)
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        bids_to_be_made = np.where(possible_prices >= expected_prices, possible_prices, 0)
        possible_revenues = np.sum(possible_volumes * bids_to_be_made, axis = 1)  
        possible_assets = [assets if possible_revenues[index] > 0 else [] for index, assets in enumerate(possible_assets)]     
        return (possible_revenues, possible_assets)
    else:
        possible_assets = [[asset for asset in hourly_assets[h] if asset.area == market.area] for h in range(len(possible_hours))]
        indices = [H.get_loc(ts) for ts in possible_hours]
        directions = np.array(dominant_directions[indices[0]:indices[-1]+1])
        mask_up = directions == "up"
        mask_down = directions == "down"
        possible_up_volumes = [sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        possible_down_volumes = [sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        #print(f"possible_up_volumes: {possible_up_volumes}")
        #print(f"possible_down_volumes: {possible_down_volumes}")
        possible_volumes = np.where(mask_up, possible_up_volumes, possible_down_volumes)
        possible_volumes = np.where(possible_volumes > [market.min_volume for _ in range(len(possible_volumes))], possible_volumes, 0)
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        bids_to_be_made = np.where(possible_prices >= expected_prices, possible_prices, 0)
        possible_revenues = np.sum(possible_volumes * bids_to_be_made, axis = 1)    
        possible_assets = [assets if possible_revenues[index] > 0 else [] for index, assets in enumerate(possible_assets)]     
    
        return (possible_revenues, possible_assets)

In [ ]:
def get_possible_bids( H : [pd.Timestamp], L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket]):
    """ Function to find the possible bids for the given input hours, assets and markets. where the possible bids are not constrained to be for every one of the 24 hours.

    Args:
        H (DateTimeIndex): list of the possible hours where bids can be placed
        L ([new_meters.PowerMeter]): list of all assets
        M ([final_markets.ReserveMarket]): list of all markets

    Returns:
        dict: dictionary with the possible bids for each market where the keys are tuples of the market name and the date and the values are tuples of the possible revenues and the assets that can bid in the market
    """
    bid_timeframe = H.append(pd.date_range(H[-1] + timedelta(hours=1), periods=48, freq='H', tz="Europe/Oslo"))
    available_assets = {hour: L.copy() for hour in bid_timeframe}
    bids = {}
    revenue = 0

    for hour in H:
        (possible_hours, market_name) = get_possible_dates(hour)
        if len(possible_hours) != 24:
            #print(f"No bids for {hour}")
            #print(f"possible_hours: {len(possible_hours)}")
            continue

        possible_markets = [m for m in M if market_name in m.name]
        for market in possible_markets:
            (possible_revenues, possible_assets) = place_bids(market=market, available_assets=available_assets, possible_hours=possible_hours)
            revenue += np.sum(possible_revenues)

            for h, bid_hour in enumerate(possible_hours):
                # Select assets with positive revenue for bidding
                assets_to_bid = [asset for asset in possible_assets[h] if possible_revenues[h] > 0]
                # Flatten the list if it's a list of lists
                #assets_to_bid = [item for sublist in assets_to_bid for item in sublist]
                # Store the bid information
                bids[(market.name, bid_hour)] = [asset.meter_id for asset in assets_to_bid]
                # Update available assets
                available_assets[bid_hour] = [asset for asset in available_assets[bid_hour] if asset not in assets_to_bid]

    return bids, revenue, available_assets
            

In [ ]:
bids, revenue, av_assets  = get_possible_bids(H[:24], L, M)